In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import os
import zipfile
from tqdm import tqdm
from collections import defaultdict
import itertools

class SmartARCSolver:
    def __init__(self):
        self.solutions = {}
        self.pattern_stats = defaultdict(int)
        
    def solve_task(self, task_data):
        """Main solver entry point"""
        examples = task_data['train']
        
        # Try solvers in order of likelihood
        solvers = [
            self.try_identity,
            self.try_constant,
            self.try_simple_transforms,
            self.try_color_operations,
            self.try_position_patterns,
            self.try_slicing,
            self.try_scaling,
            self.try_complex_patterns,
            self.try_hardcode
        ]
        
        for solver in solvers:
            code = solver(examples)
            if code and self.verify(code, examples):
                return self.minify(code)
        
        return "p=lambda g:g"
    
    def verify(self, code, examples):
        """Verify code works on examples"""
        try:
            namespace = {}
            exec(code, namespace)
            p = namespace['p']
            
            for ex in examples[:3]:  # Test on first 3
                result = p([row[:] for row in ex['input']])
                if result != ex['output']:
                    return False
            return True
        except:
            return False
    
    def minify(self, code):
        """Minimize code length"""
        # Remove unnecessary spaces
        replacements = [
            (' = ', '='), (', ', ','), (': ', ':'),
            (' + ', '+'), (' - ', '-'), (' * ', '*'),
            (' / ', '/'), (' % ', '%'), (' // ', '//'),
            (' > ', '>'), (' < ', '<'), (' == ', '=='),
            (' if ', 'if '), (' else ', 'else '),
            (' for ', 'for '), (' in ', 'in '),
            (' and ', 'and '), (' or ', 'or ')
        ]
        for old, new in replacements:
            code = code.replace(old, new)
        return code
    
    def try_identity(self, examples):
        """Check if output equals input"""
        if all(ex['input'] == ex['output'] for ex in examples):
            return "p=lambda g:g"
        return None
    
    def try_constant(self, examples):
        """Check for constant output"""
        out = examples[0]['output']
        if all(ex['output'] == out for ex in examples):
            # All same value
            if all(all(c == out[0][0] for c in row) for row in out):
                v = out[0][0]
                h, w = len(out), len(out[0])
                return f"p=lambda g:[[{v}]*{w}]*{h}"
            # Small output - hardcode
            if len(str(out)) < 60:
                return f"p=lambda g:{out}"
        return None
    
    def try_simple_transforms(self, examples):
        """Try rotations, flips, transpose"""
        transforms = [
            ("p=lambda g:g[::-1]", lambda g: g[::-1]),  # Vertical flip
            ("p=lambda g:[r[::-1]for r in g]", lambda g: [r[::-1] for r in g]),  # Horizontal flip
            ("p=lambda g:[r[::-1]for r in g[::-1]]", lambda g: [r[::-1] for r in g[::-1]]),  # 180 rotation
            ("p=lambda g:[list(r)for r in zip(*g[::-1])]", lambda g: [list(r) for r in zip(*g[::-1])]),  # 90 CW
            ("p=lambda g:[list(r)for r in zip(*g)][::-1]", lambda g: [list(r) for r in zip(*g)][::-1]),  # 90 CCW
            ("p=lambda g:[list(r)for r in zip(*g)]", lambda g: [list(r) for r in zip(*g)]),  # Transpose
        ]
        
        for code, func in transforms:
            if all(func(ex['input']) == ex['output'] for ex in examples):
                return code
        return None
    
    def try_color_operations(self, examples):
        """Try color mappings and arithmetic"""
        # Simple arithmetic operations
        operations = [
            ("p=lambda g:[[1-c for c in r]for r in g]", lambda c: 1-c),
            ("p=lambda g:[[9-c for c in r]for r in g]", lambda c: 9-c),
            ("p=lambda g:[[c%2for c in r]for r in g]", lambda c: c%2),
            ("p=lambda g:[[c%3for c in r]for r in g]", lambda c: c%3),
            ("p=lambda g:[[c%5for c in r]for r in g]", lambda c: c%5),
            ("p=lambda g:[[c*2for c in r]for r in g]", lambda c: c*2),
            ("p=lambda g:[[c+1for c in r]for r in g]", lambda c: c+1),
            ("p=lambda g:[[c//2for c in r]for r in g]", lambda c: c//2),
            ("p=lambda g:[[int(c>0)for c in r]for r in g]", lambda c: int(c>0)),
            ("p=lambda g:[[min(c+1,9)for c in r]for r in g]", lambda c: min(c+1,9)),
            ("p=lambda g:[[max(c-1,0)for c in r]for r in g]", lambda c: max(c-1,0)),
        ]
        
        for code, func in operations:
            if all([[func(c) for c in r] for r in ex['input']] == ex['output'] for ex in examples):
                return code
        
        # Color extraction
        for color in range(10):
            code = f"p=lambda g:[[c if c=={color}else 0for c in r]for r in g]"
            if self.verify(code, examples):
                return code
        
        # Color swap (for 2 colors)
        inp_colors = set(c for ex in examples for row in ex['input'] for c in row)
        if len(inp_colors) == 2:
            c1, c2 = sorted(inp_colors)
            code = f"p=lambda g:[[{c2}if c=={c1}else{c1}for c in r]for r in g]"
            if self.verify(code, examples):
                return code
        
        # Dictionary mapping
        mapping = {}
        for ex in examples:
            for i in range(len(ex['input'])):
                for j in range(len(ex['input'][0])):
                    if i < len(ex['output']) and j < len(ex['output'][0]):
                        inp_val = ex['input'][i][j]
                        out_val = ex['output'][i][j]
                        if inp_val in mapping and mapping[inp_val] != out_val:
                            mapping = None
                            break
                        mapping[inp_val] = out_val
                if mapping is None:
                    break
            if mapping is None:
                break
        
        if mapping and len(mapping) <= 4:
            map_str = str(mapping).replace(' ', '')
            code = f"p=lambda g:[[{map_str}.get(c,c)for c in r]for r in g]"
            if self.verify(code, examples):
                return code
        
        return None
    
    def try_position_patterns(self, examples):
        """Try position-based patterns"""
        out = examples[0]['output']
        h, w = len(out), len(out[0]) if out else 0
        
        # Fixed size patterns
        if all(len(ex['output']) == h and len(ex['output'][0]) == w for ex in examples):
            # Checkerboard
            colors = sorted(set(c for row in out for c in row))
            if len(colors) == 2:
                c0, c1 = colors
                if all(out[i][j] == (c0 if (i+j)%2==0 else c1) for i in range(h) for j in range(w)):
                    return f"p=lambda g:[[{c0}if(i+j)%2==0else{c1}for j in range({w})]for i in range({h})]"
                if all(out[i][j] == (c1 if (i+j)%2==0 else c0) for i in range(h) for j in range(w)):
                    return f"p=lambda g:[[{c1}if(i+j)%2==0else{c0}for j in range({w})]for i in range({h})]"
            
            # Row/column indices
            patterns = [
                (f"p=lambda g:[[i for j in range({w})]for i in range({h})]", 
                 lambda i,j: i),
                (f"p=lambda g:[[j for j in range({w})]for i in range({h})]", 
                 lambda i,j: j),
                (f"p=lambda g:[[(i+j)%10for j in range({w})]for i in range({h})]", 
                 lambda i,j: (i+j)%10),
                (f"p=lambda g:[[(i*j)%10for j in range({w})]for i in range({h})]", 
                 lambda i,j: (i*j)%10),
            ]
            
            for code, func in patterns:
                if all(out[i][j] == func(i,j) for i in range(h) for j in range(w)):
                    return code
            
            # Diagonal
            if h == w:
                diag_val = out[0][0] if out else 0
                if all(out[i][j] == (diag_val if i==j else 0) for i in range(h) for j in range(w)):
                    return f"p=lambda g:[[{diag_val}if i==j else 0for j in range({w})]for i in range({h})]"
            
            # Border
            if h >= 3 and w >= 3:
                border = out[0][0]
                inner = out[1][1] if h > 1 and w > 1 else 0
                is_border = all(out[i][j] == (border if (i==0 or i==h-1 or j==0 or j==w-1) else inner) 
                              for i in range(h) for j in range(w))
                if is_border:
                    return f"p=lambda g:[[{border}if i in[0,{h-1}]or j in[0,{w-1}]else{inner}for j in range({w})]for i in range({h})]"
        
        # Position + value operations
        ops = [
            "p=lambda g:[[(c+i)%10for j,c in enumerate(r)]for i,r in enumerate(g)]",
            "p=lambda g:[[(c+j)%10for j,c in enumerate(r)]for i,r in enumerate(g)]",
            "p=lambda g:[[(c+i+j)%10for j,c in enumerate(r)]for i,r in enumerate(g)]",
        ]
        for code in ops:
            if self.verify(code, examples):
                return code
        
        return None
    
    def try_slicing(self, examples):
        """Try slicing operations"""
        # Row slicing
        slices = [
            ("p=lambda g:g[::2]", lambda g: g[::2]),
            ("p=lambda g:g[1::2]", lambda g: g[1::2]),
            ("p=lambda g:g[::3]", lambda g: g[::3]),
            ("p=lambda g:g[1:-1]", lambda g: g[1:-1]),
        ]
        
        for code, func in slices:
            if all(func(ex['input']) == ex['output'] for ex in examples):
                return code
        
        # Column slicing
        col_slices = [
            ("p=lambda g:[r[::2]for r in g]", lambda g: [r[::2] for r in g]),
            ("p=lambda g:[r[1::2]for r in g]", lambda g: [r[1::2] for r in g]),
            ("p=lambda g:[r[::3]for r in g]", lambda g: [r[::3] for r in g]),
            ("p=lambda g:[r[1:-1]for r in g]", lambda g: [r[1:-1] for r in g]),
        ]
        
        for code, func in col_slices:
            if all(func(ex['input']) == ex['output'] for ex in examples):
                return code
        
        # Fixed size extraction
        out_h = len(examples[0]['output'])
        out_w = len(examples[0]['output'][0]) if examples[0]['output'] else 0
        
        extracts = [
            (f"p=lambda g:g[:{out_h}]", lambda g: g[:out_h]),
            (f"p=lambda g:g[-{out_h}:]", lambda g: g[-out_h:]),
            (f"p=lambda g:[r[:{out_w}]for r in g]", lambda g: [r[:out_w] for r in g]),
            (f"p=lambda g:[r[-{out_w}:]for r in g]", lambda g: [r[-out_w:] for r in g]),
        ]
        
        for code, func in extracts:
            try:
                if all(func(ex['input']) == ex['output'] for ex in examples):
                    return code
            except:
                pass
        
        # Cropping
        if out_h < len(examples[0]['input']) and out_w < len(examples[0]['input'][0]):
            code = f"p=lambda g:[r[:{out_w}]for r in g[:{out_h}]]"
            if self.verify(code, examples):
                return code
        
        return None
    
    def try_scaling(self, examples):
        """Try scaling operations"""
        # Upscaling
        scales = [
            ("p=lambda g:[[c for c in r for _ in[0,1]]for r in g for _ in[0,1]]", 2, 2),
            ("p=lambda g:[[c for c in r for _ in range(3)]for r in g for _ in range(3)]", 3, 3),
            ("p=lambda g:[r*2for r in g]*2", 2, 2),
            ("p=lambda g:[r*3for r in g]*3", 3, 3),
        ]
        
        for code, sy, sx in scales:
            valid = True
            for ex in examples:
                inp = ex['input']
                expected = []
                for row in inp:
                    for _ in range(sy):
                        new_row = []
                        for c in row:
                            new_row.extend([c] * sx)
                        expected.append(new_row)
                if expected != ex['output']:
                    valid = False
                    break
            if valid:
                return code
        
        # Downscaling
        downs = [
            ("p=lambda g:[g[i][::2]for i in range(0,len(g),2)]", 2, 2),
            ("p=lambda g:[g[i][::3]for i in range(0,len(g),3)]", 3, 3),
        ]
        
        for code, sy, sx in downs:
            if self.verify(code, examples):
                return code
        
        # Mirror operations
        mirrors = [
            ("p=lambda g:[r+r[::-1]for r in g]", lambda g: [r+r[::-1] for r in g]),
            ("p=lambda g:g+g[::-1]", lambda g: g+g[::-1]),
            ("p=lambda g:[r+r[-2::-1]for r in g]", lambda g: [r+r[-2::-1] for r in g]),
            ("p=lambda g:g+g[-2::-1]", lambda g: g+g[-2::-1]),
        ]
        
        for code, func in mirrors:
            try:
                if all(func(ex['input']) == ex['output'] for ex in examples):
                    return code
            except:
                pass
        
        return None
    
    def try_complex_patterns(self, examples):
        """Try more complex patterns"""
        out = examples[0]['output']
        h, w = len(out), len(out[0]) if out else 0
        
        # Grid patterns
        for spacing in [2, 3, 4, 5]:
            code = f"p=lambda g:[[1if i%{spacing}==0or j%{spacing}==0else 0for j in range({w})]for i in range({h})]"
            if self.verify(code, examples):
                return code
        
        # Cross pattern
        if h == w and h % 2 == 1:
            mid = h // 2
            code = f"p=lambda g:[[1if i=={mid}or j=={mid}else 0for j in range({w})]for i in range({h})]"
            if self.verify(code, examples):
                return code
        
        # Object operations
        # Gravity left
        code = "p=lambda g:[[c for c in r if c>0]+[0]*(len(r)-sum(1for c in r if c>0))for r in g]"
        if self.verify(code, examples):
            return code
        
        # Gravity right
        code = "p=lambda g:[[0]*(len(r)-sum(1for c in r if c>0))+[c for c in r if c>0]for r in g]"
        if self.verify(code, examples):
            return code
        
        # Sort rows
        code = "p=lambda g:[sorted(r)for r in g]"
        if self.verify(code, examples):
            return code
        
        # Count-based fill
        if all(len(set(c for row in ex['output'] for c in row)) == 1 for ex in examples):
            # Get the fill value
            fill_val = examples[0]['output'][0][0]
            
            # Count non-zero
            count = sum(1 for row in examples[0]['input'] for c in row if c > 0)
            if fill_val == count:
                h, w = len(out), len(out[0])
                # Build the expression carefully
                count_expr = "sum(1for r in g for c in r if c>0)"
                code = f"p=lambda g:[[{count_expr}]*{w}]*{h}"
                if self.verify(code, examples):
                    return code
        
        return None
    
    def try_hardcode(self, examples):
        """Last resort - hardcode small outputs"""
        out = examples[0]['output']
        
        # Only if output is constant and small
        if all(ex['output'] == out for ex in examples) and len(str(out)) < 80:
            return f"p=lambda g:{out}"
        
        return None


def main():
    input_dir = "/kaggle/input/google-code-golf-2025"
    output_dir = "/kaggle/working"
    submission_dir = os.path.join(output_dir, "submission")
    os.makedirs(submission_dir, exist_ok=True)
    
    solver = SmartARCSolver()
    solutions = {}
    total_chars = 0
    solved_count = 0
    
    print("🎯 Smart ARC Code Golf Solver")
    print("=============================\n")
    
    for task_num in tqdm(range(1, 401), desc="Solving tasks"):
        task_id = f"{task_num:03d}"
        task_file = os.path.join(input_dir, f"task{task_id}.json")
        
        try:
            with open(task_file, 'r') as f:
                task_data = json.load(f)
            
            code = solver.solve_task(task_data)
            solutions[task_id] = code
            total_chars += len(code)
            
            if code != "p=lambda g:g":
                solved_count += 1
            
            with open(os.path.join(submission_dir, f"task{task_id}.py"), 'w') as f:
                f.write(code)
                
        except Exception as e:
            print(f"\nError on task {task_id}: {str(e)}")
            code = "p=lambda g:g"
            solutions[task_id] = code
            total_chars += len(code)
            
            with open(os.path.join(submission_dir, f"task{task_id}.py"), 'w') as f:
                f.write(code)
    
    # Create zip file
    zip_path = os.path.join(output_dir, "submission.zip")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for task_id in solutions:
            file_path = os.path.join(submission_dir, f"task{task_id}.py")
            zf.write(file_path, f"task{task_id}.py")
    
    print(f"\n📊 Results:")
    print(f"Total characters: {total_chars:,}")
    print(f"Average per task: {total_chars/400:.1f}")
    print(f"Tasks solved: {solved_count}/400 ({solved_count/400*100:.1f}%)")
    print(f"Final score: {1_000_000 - total_chars:,}")
    
    # Show shortest solutions
    sorted_solutions = sorted([(tid, code, len(code)) for tid, code in solutions.items() 
                              if code != "p=lambda g:g"], key=lambda x: x[2])
    
    if sorted_solutions:
        print(f"\n🏆 Top 10 Shortest Solutions:")
        for i, (tid, code, length) in enumerate(sorted_solutions[:10]):
            print(f"{i+1}. Task {tid} ({length} chars): {code}")
    
    print(f"\n✅ Submission created: {zip_path}")

if __name__ == "__main__":
    main()